In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

In [2]:
path = "/home/rutvik/throwaway/memory"
files = []
os.chdir(path)
for file in os.listdir():
    if file.endswith("_MEMORY.json"):
        files.append(file)

600

In [5]:
sockets = {}
benches = set()
dses = set()
for file in files:
    parts = file.split('_')
    benches.add(bench := parts[0])
    socket = parts[1]
    dses.add(ds := parts[2])
    if socket not in sockets:
        sockets[socket] = {}
    socket = sockets[socket]
    if bench not in socket:
        socket[bench] = {}
    bench = socket[bench]
    if ds not in bench:
        bench[ds] = {}
    ds = bench[ds]
    with open(file) as fh:
        data = json.load(fh)
        for key in data.keys():
            ds[key] = data[key]

In [11]:
for socket in sockets.keys():
    os.mkdir(socket)

In [12]:
for socket in sockets:
    for bench in benches:
        data = sockets[socket][bench]
        xs = []
        dses_ = []
        ys = []
        fname = f"{socket}/{bench}_Mem_Comparison"
        print(fname)
        for ds in data:
            data_ = data[ds]
            dses_.append(ds)
            ys.append([sam / 1024 / 1024 for sam in data_["samples"]])
            num_sam = data_["numSamples"]
            frequency = data_["samplingRate"]
            xs.append(stamps := range(0, num_sam))
        for i in range(len(xs)):
            x = xs[i]
            y = ys[i]
            ds = dses_[i]
            plt.plot(x, y, label=ds)
        plt.legend()
        plt.title(fname)
        plt.xlabel("samples")
        plt.ylabel("memory consumed")
        plt.savefig(fname, dpi=400)
        plt.cla()
        plt.clf()

            
        

10/pmd_Mem_Comparison
10/zxing_Mem_Comparison
10/graphchi_Mem_Comparison
10/tradebeans_Mem_Comparison
10/fop_Mem_Comparison
10/tomcat_Mem_Comparison
10/luindex_Mem_Comparison
10/lusearch_Mem_Comparison
10/avrora_Mem_Comparison
10/tradesoap_Mem_Comparison
10/h2o_Mem_Comparison
10/eclipse_Mem_Comparison
10/sunflow_Mem_Comparison
10/biojava_Mem_Comparison
10/h2_Mem_Comparison
10/xalan_Mem_Comparison
10/batik_Mem_Comparison
10/jme_Mem_Comparison
10/jython_Mem_Comparison
10/cassandra_Mem_Comparison
8/pmd_Mem_Comparison
8/zxing_Mem_Comparison
8/graphchi_Mem_Comparison
8/tradebeans_Mem_Comparison
8/fop_Mem_Comparison
8/tomcat_Mem_Comparison
8/luindex_Mem_Comparison
8/lusearch_Mem_Comparison
8/avrora_Mem_Comparison
8/tradesoap_Mem_Comparison
8/h2o_Mem_Comparison
8/eclipse_Mem_Comparison
8/sunflow_Mem_Comparison
8/biojava_Mem_Comparison
8/h2_Mem_Comparison
8/xalan_Mem_Comparison
8/batik_Mem_Comparison
8/jme_Mem_Comparison
8/jython_Mem_Comparison
8/cassandra_Mem_Comparison
9/pmd_Mem_Comparison
9

<Figure size 432x288 with 0 Axes>